In [140]:

# utilities
import pandas as pd
import numpy as np
import os
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import pandas as pd
import pickle



# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklego.preprocessing import InformationFilter


from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score
from sklearn.metrics import classification_report
from aif360.metrics import BinaryLabelDatasetMetric

import aif360.sklearn as skm


# aif360
from aif360.sklearn.detectors import bias_scan
# Import necessary modules from aif360
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, BinaryLabelDataset


# onnx imports
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from skl2onnx import convert_sklearn
from io import BytesIO

from numpy.testing import assert_almost_equal
from onnxruntime import InferenceSession
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.datasets import load_iris
from skl2onnx.common.data_types import guess_numpy_type
from skl2onnx import to_onnx
from skl2onnx import update_registered_converter
from skl2onnx.algebra.onnx_ops import OnnxMatMul, OnnxSub




In [141]:

ds_train = pd.read_csv('./../data/train.csv')
ds_test = pd.read_csv('./../data/test.csv')

# Let's specify the features and the target
y_train = ds_train["checked"]
X_train = ds_train.drop(['checked'], axis=1)
X_train = X_train.astype(np.float32)

# Let's specify the features and the target
y_test = ds_test["checked"]
X_test = ds_test.drop(['checked'], axis=1)
X_test = X_test.astype(np.float32)

In [142]:
class BenefitsFraudDataset(BinaryLabelDataset):
    """Benefits Fraud Dataset.
    """

    def __init__(self, df, label_name='checked',
                 protected_attribute_names=['persoon_geslacht_vrouw', 'typering_hist_ind']
                 ):



        super(BenefitsFraudDataset, self).__init__(df=df, label_names=[label_name],
            protected_attribute_names=protected_attribute_names
            )


In [143]:
protected_attributes = [
    "adres_aantal_brp_adres",
    "adres_aantal_verschillende_wijken",
    "adres_aantal_verzendadres",
    "adres_aantal_woonadres_handmatig",
    "adres_dagen_op_adres",
    "adres_recentst_onderdeel_rdam",
    "adres_recentste_buurt_groot_ijsselmonde",
    "adres_recentste_buurt_nieuwe_westen",
    "adres_recentste_buurt_other",
    "adres_recentste_buurt_oude_noorden",
    "adres_recentste_buurt_vreewijk",
    "adres_recentste_plaats_other",
    "adres_recentste_plaats_rotterdam",
    "adres_recentste_wijk_charlois",
    "adres_recentste_wijk_delfshaven",
    "adres_recentste_wijk_feijenoord",
    "adres_recentste_wijk_ijsselmonde",
    "adres_recentste_wijk_kralingen_c",
    "adres_recentste_wijk_noord",
    "adres_recentste_wijk_other",
    "adres_recentste_wijk_prins_alexa",
    "adres_recentste_wijk_stadscentru",
    "adres_unieke_wijk_ratio",
    "belemmering_ind",
    "belemmering_ind_hist",
    "ontheffing_actueel_ind",
    "ontheffing_hist_ind",
    "persoon_geslacht_vrouw",
    "persoon_leeftijd_bij_onderzoek",
    "pla_historie_werk_en_inburgering",
    "relatie_kind_basisschool_kind",
    "relatie_kind_heeft_kinderen",
    "relatie_kind_huidige_aantal",
    "relatie_kind_jongvolwassen",
    "relatie_kind_leeftijd_verschil_ouder_eerste_kind",
    "relatie_kind_tiener",
    "relatie_kind_volwassen",
    "relatie_overig_actueel_vorm__ouders_verzorgers",
    "relatie_overig_kostendeler",
    "relatie_partner_aantal_partner___partner__gehuwd_",
    "relatie_partner_aantal_partner___partner__ongehuwd_",
    "relatie_partner_huidige_partner___partner__gehuwd_",
    "relatie_partner_totaal_dagen_partner",
    "typering_hist_ind",
    "typering_ind"
]

In [144]:
ds_train = pd.read_csv('./../data/train.csv')
ds_test = pd.read_csv('./../data/test.csv')

# Let's specify the features and the target
y_train = ds_train["checked"]
X_train = ds_train.drop(['checked'], axis=1)
X_train = X_train.astype(np.float32)

# Let's specify the features and the target
y_test = ds_test["checked"]
X_test = ds_test.drop(['checked'], axis=1)
X_test = X_test.astype(np.float32)


# X_fair = InformationFilter(protected_attributes).fit_transform(ds_train)
# X_fair = pd.DataFrame(X_fair,
#                       columns=[n for n in ds_train.columns if n not in protected_attributes])

# # Let's specify the features and the target
# y_fair = X_fair["checked"]
# X_fair = X_fair.drop(['checked'], axis=1)
# X_fair = X_fair.astype(np.float32)


# print('fair', X_fair.shape)


In [145]:

class InformationFilterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, protected_attributes):
        self.protected_attributes = protected_attributes
    
            
    def fit(self, X, y=None):
        # Fit your transformer to the data
        # If no fitting is needed, you can just return self
        return self

    def transform(self, X):
        # Perform the transformation on the input data X
        # Return the transformed data
        X_fair = InformationFilter(self.protected_attributes).fit_transform(X)
        X_fair = pd.DataFrame(X_fair,
                        columns=[n for n in ds_train.columns if n not in protected_attributes])
        # Let's specify the features and the target
        # y_fair = X_fair["checked"]
        X_fair = X_fair.drop(['checked'], axis=1)
        X_fair = X_fair.astype(np.float32)
        print(X_fair)
        self.coef_ = X_fair
        return X_fair
        
        ...

In [146]:
# Define a gradient boosting classifier
classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

In [152]:



# Create a pipeline object with our selector and classifier
# NOTE: You can create custom pipeline objects but they must be registered to onnx or it will not recognise them
# Because of this we recommend using the onnx known objects as defined in the documentation
pipeline = Pipeline(steps=[('filter', InformationFilterTransformer(protected_attributes=protected_attributes)), ('classifier', classifier)])

In [153]:
# Let's train a simple model


pipeline.fit(ds_train, y_train)

# Let's evaluate the model
y_pred = pipeline.predict(ds_test)

original_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the original model: ', original_accuracy)





d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:109: RuntimeWarning: invalid value encountered in scalar divide
  return vec.dot(unto) / unto.dot(unto)
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:223: RuntimeWarning: invalid value encountered in cast
  X_fair[:, i] = X_fair[:, i] - vector_projection(X_fair[:, i], v)


       afspraak_aanmelding_afgesloten  afspraak_aantal_woorden  \
0                       -9.223372e+18            -9.223372e+18   
1                       -9.223372e+18            -9.223372e+18   
2                       -9.223372e+18            -9.223372e+18   
3                       -9.223372e+18            -9.223372e+18   
4                       -9.223372e+18            -9.223372e+18   
...                               ...                      ...   
10111                   -9.223372e+18            -9.223372e+18   
10112                   -9.223372e+18            -9.223372e+18   
10113                   -9.223372e+18            -9.223372e+18   
10114                   -9.223372e+18            -9.223372e+18   
10115                   -9.223372e+18            -9.223372e+18   

       afspraak_afgelopen_jaar_afsprakenplan  \
0                              -9.223372e+18   
1                              -9.223372e+18   
2                              -9.223372e+18   
3              

d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:109: RuntimeWarning: invalid value encountered in scalar divide
  return vec.dot(unto) / unto.dot(unto)
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:223: RuntimeWarning: invalid value encountered in cast
  X_fair[:, i] = X_fair[:, i] - vector_projection(X_fair[:, i], v)


      afspraak_aanmelding_afgesloten  afspraak_aantal_woorden  \
0                      -9.223372e+18            -9.223372e+18   
1                      -9.223372e+18            -9.223372e+18   
2                      -9.223372e+18            -9.223372e+18   
3                      -9.223372e+18            -9.223372e+18   
4                      -9.223372e+18            -9.223372e+18   
...                              ...                      ...   
2524                   -9.223372e+18            -9.223372e+18   
2525                   -9.223372e+18            -9.223372e+18   
2526                   -9.223372e+18            -9.223372e+18   
2527                   -9.223372e+18            -9.223372e+18   
2528                   -9.223372e+18            -9.223372e+18   

      afspraak_afgelopen_jaar_afsprakenplan  \
0                             -9.223372e+18   
1                             -9.223372e+18   
2                             -9.223372e+18   
3                             -

In [154]:
"""
BIAS SCAN: See results/bias_scan_results.txt for results
"""

def get_bias_scan(X_train, y_train, y_train_pred):
    bias_scan_results = bias_scan(X_train, y_train, y_train_pred)
    return bias_scan_results

# Let's evaluate the model
y_train_pred = pipeline.predict(ds_train)
y_train_pred = pd.Series(y_train_pred)

get_bias = False
if get_bias == True:
    bias_scan_results = get_bias_scan(X_train, y_train, y_train_pred)


d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:109: RuntimeWarning: invalid value encountered in scalar divide
  return vec.dot(unto) / unto.dot(unto)
d:\tudelft\test-val-for-ai-project\.env\Lib\site-packages\sklego\preprocessing\projections.py:223: RuntimeWarning: invalid value encountered in cast
  X_fair[:, i] = X_fair[:, i] - vector_projection(X_fair[:, i], v)


       afspraak_aanmelding_afgesloten  afspraak_aantal_woorden  \
0                       -9.223372e+18            -9.223372e+18   
1                       -9.223372e+18            -9.223372e+18   
2                       -9.223372e+18            -9.223372e+18   
3                       -9.223372e+18            -9.223372e+18   
4                       -9.223372e+18            -9.223372e+18   
...                               ...                      ...   
10111                   -9.223372e+18            -9.223372e+18   
10112                   -9.223372e+18            -9.223372e+18   
10113                   -9.223372e+18            -9.223372e+18   
10114                   -9.223372e+18            -9.223372e+18   
10115                   -9.223372e+18            -9.223372e+18   

       afspraak_afgelopen_jaar_afsprakenplan  \
0                              -9.223372e+18   
1                              -9.223372e+18   
2                              -9.223372e+18   
3              

In [156]:


# Get the final estimator (model) from the pipeline
final_model = pipeline.named_steps['classifier']

# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    final_model, initial_types=[('X', FloatTensorType((None, X_train.shape[1])))],
    target_opset=12)

# Let's save the model
onnx.save(onnx_model, "./../model/good_model.onnx")

# Let's load the model
new_session = rt.InferenceSession("./../model/good_model.onnx")

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)


Accuracy of the ONNX model:  0.8936338473705022
